# Visual Search demo

<img src="logo.png" width=350>

## 4. Webapp
Build a visual search web app using Azure AI Vision and Azure AI Search by embedding images and prompts into vectors, indexing them, and retrieving similar results via Gradio UI.

## Steps
- Generate vector embeddings for images and text using Azure AI.
- Create and configure an Azure AI Search index.
- Upload images to Azure Blob Storage.
- Index the image vectors into Azure AI Search.
- Build a Gradio web app for prompt-based and image-based search.
- Launch the app with a tabbed interface for both search modes and image generation.

## Visual search with vector embeddings
Vector embeddings are a way of representing content such as text or images as vectors of real numbers in a high-dimensional space. These embeddings are often learned from large amounts of textual and visual data using machine learning algorithms like neural networks. Each dimension of the vector corresponds to a different feature or attribute of the content, such as its semantic meaning, syntactic role, or context in which it commonly appears. By representing content as vectors, we can perform mathematical operations on them to compare their similarity or use them as inputs to machine learning models.

## Business applications
- Digital asset management: Image retrieval can be used to manage large collections of digital images, such as in museums, archives, or online galleries. Users can search for images based on visual features and retrieve the images that match their criteria.
- Medical image retrieval: Image retrieval can be used in medical imaging to search for images based on their diagnostic features or disease patterns. This can help doctors or researchers to identify similar cases or track disease progression.
- Security and surveillance: Image retrieval can be used in security and surveillance systems to search for images based on specific features or patterns, such as in, people & object tracking, or threat detection.
- Forensic image retrieval: Image retrieval can be used in forensic investigations to search for images based on their visual content or metadata, such as in cases of cyber-crime.
- E-commerce: Image retrieval can be used in online shopping applications to search for similar products based on their features or descriptions or provide recommendations based on previous purchases.
- Fashion and design: Image retrieval can be used in fashion and design to search for images based on their visual features, such as color, pattern, or texture. This can help designers or retailers to identify similar products or trends.

## Images
In this notebook we took some samples fashion images are taken from this link:<br>
https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-dataset

<img src="screenshot.jpg">

In [1]:
import base64
import datetime
import gradio as gr
import io
import json
import os
import re
import requests
import sys

from azurevisualsearch import get_image_embedding, get_index_stats, text_embedding
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.storage.blob import BlobServiceClient
from azure.search.documents.models import VectorizedQuery
from dotenv import load_dotenv
from io import BytesIO
from PIL import Image
from typing import List, Tuple, Optional

In [2]:
print(f"Today is {datetime.datetime.now().strftime('%Y-%m-%d')}")

Today is 2025-07-30


In [3]:
print(f"Python version: {sys.version}")

Python version: 3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]


## 1. Azure AI Services

In [4]:
load_dotenv("azure.env")

# Azure AI Vision
azure_vision_key = os.getenv("azure_vision_key")
azure_vision_endpoint = os.getenv("azure_vision_endpoint")
api_version = "2024-02-01"
model_version = "2023-04-15"

# Azure AI Search
azure_search_key = os.getenv("azure_search_key")
azure_search_endpoint = os.getenv("azure_search_endpoint")
index_name = "fashion-demo-2025"

# Azure storage account
blob_connection_string = os.getenv("blob_connection_string")
container_name = os.getenv("container_name")

In [5]:
IMAGES_DIR = "images"

COMPOSE_DIR = f"{IMAGES_DIR}/compose_images"

## 2. Blob storage

In [6]:
# Connect to Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
container_client = blob_service_client.get_container_client(container_name)
blobs = container_client.list_blobs()

first_blob = next(blobs)
blob_url = container_client.get_blob_client(first_blob).url
print(f"URL of the first blob: {blob_url}")

URL of the first blob: https://azurestorageaccountsr.blob.core.windows.net/fashionimages/fashion/0390469004.jpg


## 3. Azure AI Search

In [7]:
try:
    # Setting the Azure AI Search client
    print("Setting the Azure AI Search client")
    
    search_client = SearchClient(endpoint=azure_search_endpoint,
                                 index_name=index_name,
                                 credential=AzureKeyCredential(azure_search_key)
                                )
    print("Done")

except:
    print(f"❌ Request failed. Cannot create Azure AI Search client: {azure_search_endpoint}")

Setting the Azure AI Search client
Done


In [8]:
document_count, storage_size = get_index_stats(index_name)

📡 Azure AI Search index status for: fashion-demo-2025

{
  "@odata.context": "https://azureaisearch-sr.search.windows.net/$metadata#Microsoft.Azure.Search.V2024_07_01.IndexStatistics",
  "documentCount": 10226,
  "storageSize": 115931376,
  "vectorIndexSize": 42451784
}


In [9]:
print(f"🔢 Number of documents in the index = {document_count:,}")
print(f"📏 Size of the index = {storage_size / (1024 * 1024):.2f} MB")

🔢 Number of documents in the index = 10,226
📏 Size of the index = 110.56 MB


## 4. Webapp

In [10]:
topn = 5
imgsize = 512

footnote = "✨ Powered by Azure AI Foundry, Azure AI Vision and Azure AI Search."
header_prompt = "🔍 Visual Search with Azure AI using a prompt"
header_images = "📸 Visual Search with Azure AI using an image"

#logo_image = "https://github.com/retkowsky/images/blob/master/banni%C3%A8re.jpg?raw=true"
#image = "<center> <img src= {} width=1024px></center>".format(logo_image)

## 4.1 Prompt webapp

In [11]:
def webapp_prompt_fn(prompt: str, topn: int = 5) -> List[Tuple[str, str]]:
    """
    Handles the prompt-based image search for the Gradio web application.

    Parameters:
    - prompt (str): The natural language query describing the desired image content.
    - topn (int, optional): The number of top matching results to return. Defaults to 5.

    Returns:
    - List[Tuple[str, str]]: A list of tuples where each tuple contains:
        - The file path to the matched image.
        - A formatted string with the image filename and its similarity score.
    
    Notes:
    - This function relies on `prompt_search_gradio` to retrieve the top matching image paths,
      their similarity scores, and filenames based on the input prompt.
    """
    # Assuming prompt_search_gradio can return file paths
    file_paths, scores, filenames = prompt_search_gradio(prompt, topn)
    
    return [
        (file_path, f"Image: {filename}\nwith score = {score:.5f}")
        for file_path, score, filename in zip(file_paths, scores, filenames)
    ]


prompt_examples = [
    ["T-shirt with 'Paris' text on it", 3],
    ["Some torn blue jeans", 3],
    ["I want some sneakers with birds on it", 3],
    ["I want a beautiful hat.", 3],
    ["Ray ban with leopard frames", 3],
]


with gr.Blocks(title="🖼️ Visual Search WebApp") as webapp_prompt:
    gr.Markdown(f"## {header_prompt}")
    #gr.Markdown(image)

    with gr.Row():
        prompt_input = gr.Textbox(
            lines=2,
            label="🔍 What do you want to search?",
            placeholder="💡Enter your prompt to search the products database..."
        )
        topn_slider = gr.Slider(minimum=1,
                                maximum=10,
                                value=3,
                                step=1,
                                label="📊 Top N")

    submit_button = gr.Button("🚀 SEARCH IMAGES")

    # Output Gallery shown AFTER the button
    gallery_output = gr.Gallery(label="🎯 Search results",
                                columns=3,
                                height="auto")

    # Set up the interaction
    submit_button.click(fn=webapp_prompt_fn,
                        inputs=[prompt_input, topn_slider],
                        outputs=gallery_output)

    gr.Markdown("### 💡 Example prompts")
    gr.Examples(examples=prompt_examples, inputs=[prompt_input, topn_slider])

    gr.Markdown(footnote)

In [12]:
def prompt_search_gradio(
        prompt: str,
        topn: int = 5) -> Tuple[List[Image.Image], List[float], List[str]]:
    """
    Performs a semantic search on an Azure AI Search index using a text prompt and returns the top matching images.

    Parameters:
    - prompt (str): The natural language query used to search for similar images.
    - topn (int, optional): The number of top results to retrieve. Defaults to 5.

    Returns:
    - Tuple[List[Image.Image], List[float], List[str]]:
        - A list of PIL Image objects corresponding to the top search results.
        - A list of similarity scores (floats) for each result.
        - A list of filenames (str) for the matched images.

    Behavior:
    - Converts the input prompt into a vector using `text_embedding`.
    - Queries the Azure AI Search index using vector similarity search.
    - Retrieves the top matching image filenames and their similarity scores.
    - Downloads and resizes the corresponding images from Azure Blob Storage.

    Notes:
    - If an image fails to load, it is skipped with an error message.
    - Any exceptions during the search process are caught and logged.
    """
    images_list = []
    scores_list = []
    search_results = []

    try:
        # Azure Search client setup
        search_client = SearchClient(endpoint=azure_search_endpoint,
                                     index_name=index_name,
                                     credential=AzureKeyCredential(azure_search_key)
                                    )

        query_vector = text_embedding(prompt)

        response = search_client.search(search_text=None,
                                        vector_queries=[
                                            VectorizedQuery(
                                                vector=query_vector,
                                                k_nearest_neighbors=topn,
                                                fields="imagevector")
                                        ])

        for idx, doc in enumerate(response, start=1):
            filename = doc.get("imagefile")
            score = doc.get("@search.score", 0.0)

            if filename:
                print(f"Top {idx:02}: {filename} with Cosine Similarity = {score:.5f}")
                search_results.append(filename)
                scores_list.append(score)

        for filename in search_results:
            try:
                blob_client = container_client.get_blob_client(filename)
                blob_data = blob_client.download_blob().readall()
                img = Image.open(io.BytesIO(blob_data)).resize((imgsize, imgsize))
                images_list.append(img)
            except Exception as img_err:
                print(f"Failed to load image {filename}: {img_err}")

    except Exception as e:
        print(f"Image search failed: {e}")

    return images_list, scores_list, search_results

In [13]:
#webapp_prompt.launch(share=True)

### 4.2 Image webapp

In [14]:
def image_search_gradio(
        imagefile: str,
        topn: int = 5) -> Tuple[List[Image.Image], List[float], List[str]]:
    """
    Performs a reverse image search using Azure AI Search and returns the top visually similar images.

    Parameters:
    - imagefile (str): The path or identifier of the input image to search with.
    - topn (int, optional): The number of top similar images to retrieve. Defaults to 5.

    Returns:
    - Tuple[List[Image.Image], List[float], List[str]]:
        - A list of PIL Image objects representing the top matching images.
        - A list of similarity scores (floats) for each result.
        - A list of filenames (str) corresponding to the matched images.

    Behavior:
    - Computes the embedding vector of the input image using `get_image_embedding`.
    - Queries the Azure AI Search index using vector similarity search.
    - Retrieves the top matching image filenames and their similarity scores.
    - Downloads and resizes the matched images from Azure Blob Storage.

    Notes:
    - If an image fails to load, it is skipped with an error message.
    - Any exceptions during the search or image retrieval process are caught and logged.
    """
    images_list = []
    scores_list = []
    search_results = []

    try:
        # Azure Search client setup
        search_client = SearchClient(endpoint=azure_search_endpoint,
                                     index_name=index_name,
                                     credential=AzureKeyCredential(azure_search_key)
                                    )

        query_vector = get_image_embedding(imagefile)

        response = search_client.search(search_text=None,
                                        vector_queries=[
                                            VectorizedQuery(
                                                vector=query_vector,
                                                k_nearest_neighbors=topn,
                                                fields="imagevector")
                                        ])

        for idx, doc in enumerate(response, start=1):
            filename = doc.get("imagefile")
            score = doc.get("@search.score", 0.0)

            if filename:
                print(f"Top {idx:02}: {filename} with Cosine Similarity = {score:.5f}")
                search_results.append(filename)
                scores_list.append(score)

        for filename in search_results:
            try:
                blob_client = container_client.get_blob_client(filename)
                blob_data = blob_client.download_blob().readall()
                img = Image.open(io.BytesIO(blob_data)).resize((imgsize, imgsize))
                images_list.append(img)
            except Exception as img_err:
                print(f"Failed to load image {filename}: {img_err}")

    except Exception as e:
        print(f"Image search failed: {e}")

    return images_list, scores_list, search_results

In [15]:
def webapp_image_fn(prompt: str,
                    topn: int = 5) -> List[Tuple[Image.Image, str]]:
    """
    Handles reverse image search for the Gradio web application using an uploaded image file path.

    Parameters:
    - prompt (str): The file path of the uploaded image used as the search query.
    - topn (int, optional): The number of top similar images to retrieve. Defaults to 5.

    Returns:
    - List[Tuple[Image.Image, str]]: A list of tuples, each containing:
        - A PIL Image object representing a similar image.
        - A formatted string with the image filename and its similarity score.

    Notes:
    - This function wraps the `image_search_gradio` function to format the results for display in a Gradio Gallery component.
    - The `prompt` parameter is expected to be a valid image file path, not a text prompt.
    """
    images, scores, filenames = image_search_gradio(prompt, topn)
    
    return [(img, f"Image: {filename}\n with score = {score:.5f}")
            for img, score, filename in zip(images, scores, filenames)]


images_examples = [
    ["sources/image1.jpg", 3],
    ["sources/image2.jpg", 3],
    ["sources/image3.jpg", 3],
    ["sources/image4.jpg", 3],
    ["sources/image5.jpg", 3],
]


with gr.Blocks(title="🖼️ Visual Search WebApp") as webapp_image:
    gr.Markdown(f"## {header_images}")
    #gr.Markdown(image)

    with gr.Row():
        prompt_input = gr.File(label="🔍 Select an image")
        topn_slider = gr.Slider(minimum=1,
                                maximum=10,
                                value=3,
                                step=1,
                                label="📊 Top N")

    # Display the uploaded image
    uploaded_image_display = gr.Image(label="📥 Uploaded image",
                                      type="filepath")

    # Update the image display when a file is uploaded
    prompt_input.change(fn=lambda f: f.name if f else None,
                        inputs=prompt_input,
                        outputs=uploaded_image_display)

    submit_button = gr.Button("🚀 SEARCH SIMILAR IMAGES")

    gallery_output = gr.Gallery(label="🎯 Search results",
                                columns=3,
                                height="auto")

    submit_button.click(fn=webapp_image_fn,
                        inputs=[prompt_input, topn_slider],
                        outputs=gallery_output)

    gr.Markdown("### 💡 Example prompts")
    gr.Examples(examples=images_examples, inputs=[prompt_input, topn_slider])

    gr.Markdown(footnote)

In [16]:
#webapp_image.launch(share=True)

### 4.3 Compose images

In [17]:
gen_prompt_examples = [
    [
        "Create a photorealistic image of a man. He is wearing the 4 clothing items shown in the reference images. This man is walking along a busy city street, with the Eiffel Tower clearly visible in the background. A German Shepherd dog is walking closely beside him."
    ],
    [
        "Create a photorealistic image of a man. He is wearing the 4 clothing items shown in the reference images. This man is walking along a quiet city street, close to a park, with the Sydney Opera House visible in the background. A Labrador retriever dog is walking closely beside him."
    ],
    [
        "Create a photorealistic image of a man. He is wearing the 4 clothing items shown in the reference images. This man is sitting on a chair using a Windows laptop in a cosy apartment. A cat is on the ground. We can see the rain outside from the window."
    ],
    [
        "Create a photorealistic image of a man. He is wearing the 4 clothing items shown in the reference images. He is walking close to a swimming pool. A young child walks closely beside him. The sky in the background should appear sunny."
    ],
    [
        "Create a Studio Ghibli image of a man. He is wearing the 4 clothing items shown in the reference images. The man is close to a pink Rolls-Royce car."
    ],
    [
        "Create an image in the style of Edward Hopper of a man. He is wearing the 4 clothing items shown in the reference images. This man is outside a pub during a rainy day. He is waiting outside the pub with a couple of friends, close to a red phone booth."
    ],
    [
        "Combine the images to generate an avatar. Use cartoon style."
    ],
]

In [18]:
def image_compose_fn(source_images: List[str],
                     prompt: str,
                     n: int = 1,
                     size: str = "1024x1024",
                     quality: str = "high") -> Optional[List[str]]:
    """
    Sends a set of source images and a text prompt to an Azure OpenAI image composition endpoint
    to generate new AI-composed images.

    Parameters:
    - source_images (List[str]): List of file paths to the source images to be composed.
    - prompt (str): A natural language description guiding how the images should be composed.
    - n (int, optional): Number of composed images to generate. Defaults to 1.

    Returns:
    - Optional[List[str]]: A list of file paths to the generated images if successful, or None if an error occurs.

    Behavior:
    - Validates the Azure OpenAI endpoint and API key from environment variables.
    - Constructs a POST request to the Azure OpenAI image composition API with the provided images and prompt.
    - Decodes the base64-encoded image responses and saves them as JPEG files in a `generated_images` directory.
    - Returns the file paths of the saved images.

    Notes:
    - Automatically creates the output directory if it doesn't exist.
    - Handles and logs exceptions during the request or image processing steps.
    - Closes all file handles after the request is made.
    """
    try:
        IMAGESGEN_DIR = "generated_images"
        os.makedirs(IMAGESGEN_DIR, exist_ok=True)

        files = [("image[]", open(image, "rb")) for image in source_images]
        headers = {"api-key": os.getenv("key")}
        endpoint = os.getenv("endpoint")

        if not endpoint:
            raise ValueError(
                "AZURE_OPENAI_ENDPOINT environment variable not set")

        aoai_name = re.search(r'https://(.*?)/openai', endpoint).group(1)
        url = f"https://{aoai_name}/openai/deployments/gpt-image-1/images/edits?api-version=2025-04-01-preview"

        data = {
            "prompt": prompt,
            "n": n,  # Number of images to generate
            "size": size,   # Options: 1024x1024, 1536x1024, 1024x1536
            "quality": quality, # option: low, medium, high
            "output_compression": 100,
            "output_format": "jpeg",
        }

        response = requests.post(url, headers=headers, files=files, data=data)
        response.raise_for_status()
        images_data = response.json()["data"]
        encoded_images = [img["b64_json"] for img in images_data]

        # Close file handles
        for _, file_handle in files:
            file_handle.close()

        # Parse generated images
        output_images_list = []
        
        for encoded_image in encoded_images:
            img = Image.open(BytesIO(base64.b64decode(encoded_image)))
            # Save image to file
            now = str(datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3])
            output_file = os.path.join(IMAGESGEN_DIR, f"compose_{now}.jpg")
            img.save(output_file)
            print(f"✅ Generated AI image file is saved: {output_file}")
            output_images_list.append(output_file)

        return output_images_list

    except Exception as e:
        print(f"❌ Error generating images: {e}")
        return None

In [19]:
def process_uploaded_images(files):
    """
    Processes a list of uploaded image files for preview and returns their paths and a status message.

    Parameters:
    - files (List[UploadedFile]): A list of uploaded file objects (e.g., from a Gradio File component).

    Returns:
    - Tuple[List[PIL.Image.Image], str]:
        - A list of PIL Image objects for preview display.
        - A status message indicating the number of successfully uploaded images or an error message if none were uploaded.

    Behavior:
    - Checks if any files were uploaded.
    - For each valid file:
        - Extracts the file path.
        - Loads the image using PIL for preview.
    - Returns the list of preview images and a success message.
    """
    if not files:
        return [], "No images uploaded"

    image_paths = []
    preview_images = []

    for file in files:
        if file is not None:
            # Copy uploaded file to temporary location
            temp_path = file.name
            image_paths.append(temp_path)
            # Load image for preview
            img = Image.open(temp_path)
            preview_images.append(img)

    status = f"✅ Uploaded {len(image_paths)} images successfully"
    
    return preview_images, status

In [20]:
def generate_composed_images(uploaded_files, prompt, num_images, size, quality):
    """
    Generates new images based on uploaded input images and a text prompt using an image composition function.

    Parameters:
    - uploaded_files (List[UploadedFile]): A list of uploaded image files (e.g., from a Gradio File component).
    - prompt (str): A text prompt describing the desired transformation or composition.
    - num_images (int): The number of composed images to generate.
    - size (str): 1024x1024, 1536x1024, 1024x1536
    - quality (str): low, medium, high
    
    Returns:
    - Tuple[List[PIL.Image.Image], str]:
        - A list of generated PIL Image objects.
        - A status message indicating success or failure.

    Behavior:
    - Validates that at least one image is uploaded and a prompt is provided.
    - Extracts file paths from the uploaded files.
    - Calls `image_compose_fn` to generate new images based on the input.
    - Loads the generated images for display.
    - Handles and reports any errors during the generation process.
    """
    if not uploaded_files:
        return [], "❌ Please upload at least one image first"

    if not prompt.strip():
        return [], "❌ Please enter a prompt"

    # Extract file paths from uploaded files
    image_paths = [file.name for file in uploaded_files if file is not None]

    if not image_paths:
        return [], "❌ No valid images found"

    # Call your image composition function
    try:
        result_paths = image_compose_fn(image_paths, prompt, int(num_images), size=size, quality=quality)

        if result_paths:
            # Load generated images for display
            generated_images = []
            for path in result_paths:
                img = Image.open(path)
                generated_images.append(img)

            status = f"✅ Successfully generated {len(generated_images)} images"
            return generated_images, status
        else:
            return [], "❌ Failed to generate images"

    except Exception as e:
        return [], f"❌ Error during generation: {str(e)}"


In [21]:
def create_gradio_app():
    """
    Creates and returns a Gradio web application for AI-based image composition.

    Returns:
    - gr.Blocks: A configured Gradio Blocks interface for uploading images, entering prompts, and generating composed images.

    Features:
    - Users can upload multiple images.
    - A text prompt guides the AI on how to compose the uploaded images.
    - A slider allows users to choose how many composed images to generate.
    - Displays previews of uploaded images and the generated results.
    - Includes example prompts to help users get started.

    Components:
    - File upload input for multiple images.
    - Textbox for composition prompt.
    - Slider to select number of images to generate.
    - Button to trigger image generation.
    - Galleries to preview uploaded and generated images.
    - Status messages for upload and generation steps.
    - Example prompts section for inspiration.

    Notes:
    - Uses `process_uploaded_images` to handle file uploads.
    - Uses `generate_composed_images` to generate new images based on the prompt and uploaded files.
    """
    with gr.Blocks(title="AI Image Composer",
                   theme=gr.themes.Soft()) as webapp_genimage:
        
        gr.Markdown("# 🎨 AI Image Composer")
        gr.Markdown("Upload multiple images and use AI to compose them into new creations!")
        #gr.Markdown(image)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 📁 1. Upload images")
                uploaded_files = gr.File(label="Select Images to use",
                                         file_count="multiple",
                                         file_types=["image"],
                                         type="filepath")

                upload_status = gr.Textbox(label="Images upload status",
                                           interactive=False,
                                           lines=1)

                gr.Markdown("### ✏️ 2. Composition prompt")
                prompt_input = gr.Textbox(
                    label="Prompt",
                    placeholder="Describe how you want the images to be composed...",
                    lines=3)

                images_resolution = gr.Radio(
                    label="Resolution of the images to generate with gpt-image-1",
                    choices=["1024x1024", "1536x1024", "1024x1536"],
                    value="1024x1024",
                )

                images_quality = gr.Radio(
                    label="Quality of the images to generate with gpt-image-1",
                    choices=["low", "medium", "high"],
                    value="high",
                )

                images_number = gr.Slider(
                    label="Nomber of the images to generate with gpt-image-1",
                    minimum=1,
                    maximum=5,
                    value=3,
                    step=1,
                )

                generate_btn = gr.Button("🎨 GENERATED COMPOSED IMAGES",
                                         variant="primary",
                                         size="lg")

                generation_status = gr.Textbox(label="Generation status",
                                               interactive=False,
                                               lines=1)

            with gr.Column(scale=2):
                gr.Markdown("### 👀 Uploaded images preview")
                uploaded_gallery = gr.Gallery(label="Uploaded images",
                                              show_label=True,
                                              elem_id="uploaded_gallery",
                                              columns=3,
                                              rows=2,
                                              height="500px")

                gr.Markdown("### 🎯 Generated images")
                generated_gallery = gr.Gallery(label="Composed images",
                                               show_label=True,
                                               elem_id="generated_gallery",
                                               columns=2,
                                               rows=2,
                                               height="700px")

        # Event handlers
        uploaded_files.change(fn=process_uploaded_images,
                              inputs=[uploaded_files],
                              outputs=[uploaded_gallery, upload_status])

        generate_btn.click(fn=generate_composed_images,
                           inputs=[uploaded_files, prompt_input, images_number, images_resolution, images_quality],
                           outputs=[generated_gallery, generation_status])

        # Add example section
        gr.Markdown("### 💡 Example prompts")
        gr.Examples(
            examples=gen_prompt_examples,
            inputs=[prompt_input],
        )
        gr.Markdown(footnote)

    return webapp_genimage


webapp_genimage = create_gradio_app()

In [22]:
#webapp_genimage.launch(share=True)

### Final webapp: Combining the three gradio apps into one

In [23]:
professional_css = """
/* Main theme and layout */
body {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    margin: 0;
    padding: 0;
}

/* Main container styling */
.gradio-container {
    max-width: 1400px;
    margin: 0 auto;
    background: rgba(255, 255, 255, 0.95);
    border-radius: 20px;
    box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
    backdrop-filter: blur(10px);
    margin-top: 20px;
    margin-bottom: 20px;
}

/* Header styling */
.app-header {
    text-align: center;
    padding: 2rem 0;
    background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
    border-radius: 20px 20px 0 0;
    color: white;
    margin-bottom: 2rem;
}

.app-title {
    font-size: 2.5rem;
    font-weight: 700;
    margin-bottom: 0.5rem;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);
}

.app-subtitle {
    font-size: 1.2rem;
    opacity: 0.9;
    font-weight: 300;
}

/* Tab styling */
.tab-nav {
    background: #f8f9fa;
    border-radius: 15px;
    padding: 1rem;
    margin-bottom: 2rem;
    box-shadow: inset 0 2px 4px rgba(0, 0, 0, 0.06);
}

.tab-nav button {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    border: none;
    color: white;
    padding: 1rem 2rem;
    border-radius: 12px;
    font-weight: 600;
    font-size: 1rem;
    transition: all 0.3s ease;
    margin: 0 0.5rem;
    box-shadow: 0 4px 15px rgba(102, 126, 234, 0.4);
}

/* Make Generated Images tab bigger and more prominent */
.tab-nav button:nth-child(3) {
    padding: 1.5rem 3rem;
    font-size: 1.2rem;
    font-weight: 700;
    background: linear-gradient(135deg, #ff6b6b 0%, #ee5a24 100%);
    box-shadow: 0 6px 20px rgba(255, 107, 107, 0.4);
    transform: scale(1.1);
    border: 2px solid rgba(255, 255, 255, 0.3);
}

.tab-nav button:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 20px rgba(102, 126, 234, 0.6);
}

.tab-nav button:nth-child(3):hover {
    transform: scale(1.15) translateY(-3px);
    box-shadow: 0 8px 25px rgba(255, 107, 107, 0.6);
}

.tab-nav button.selected {
    background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
    transform: translateY(-1px);
}

.tab-nav button:nth-child(3).selected {
    background: linear-gradient(135deg, #ff9ff3 0%, #f368e0 100%);
    transform: scale(1.15) translateY(-2px);
    box-shadow: 0 10px 30px rgba(243, 104, 224, 0.5);
}

/* Generated Images section specific styling */
.tab-content:nth-child(3) {
    padding: 3rem;
    background: linear-gradient(135deg, rgba(255, 107, 107, 0.05) 0%, rgba(238, 90, 36, 0.05) 100%);
    border-radius: 20px;
    margin: 1rem 0;
    min-height: 80vh;
}

.tab-content:nth-child(3) .input-container {
    padding: 3rem;
    border: 3px solid rgba(255, 107, 107, 0.3);
    background: linear-gradient(135deg, rgba(255, 255, 255, 0.95) 0%, rgba(255, 249, 249, 0.95) 100%);
}

.tab-content:nth-child(3) .results-container {
    padding: 3rem;
    border-left: 6px solid #ff6b6b;
    background: linear-gradient(135deg, rgba(255, 255, 255, 0.98) 0%, rgba(255, 249, 249, 0.98) 100%);
    min-height: 60vh;
}

/* Make generated images display larger */
.tab-content:nth-child(3) img {
    max-width: 100%;
    border-radius: 20px;
    box-shadow: 0 15px 35px rgba(255, 107, 107, 0.3);
    transition: all 0.3s ease;
}

.tab-content:nth-child(3) img:hover {
    transform: scale(1.02);
    box-shadow: 0 20px 40px rgba(255, 107, 107, 0.4);
}

/* Input styling */
.input-container {
    background: white;
    border-radius: 15px;
    padding: 2rem;
    margin-bottom: 2rem;
    box-shadow: 0 8px 25px rgba(0, 0, 0, 0.08);
    border: 1px solid rgba(102, 126, 234, 0.1);
}

input[type="text"], textarea {
    border: 2px solid #e9ecef;
    border-radius: 12px;
    padding: 1rem;
    font-size: 1rem;
    transition: all 0.3s ease;
    background: #f8f9fa;
}

input[type="text"]:focus, textarea:focus {
    border-color: #4facfe;
    background: white;
    box-shadow: 0 0 0 3px rgba(79, 172, 254, 0.1);
    outline: none;
}

/* Button styling */
.btn-primary {
    background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
    border: none;
    color: white;
    padding: 1rem 2rem;
    border-radius: 12px;
    font-weight: 600;
    font-size: 1.1rem;
    cursor: pointer;
    transition: all 0.3s ease;
    box-shadow: 0 4px 15px rgba(79, 172, 254, 0.4);
}

.btn-primary:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 20px rgba(79, 172, 254, 0.6);
}

/* Results container */
.results-container {
    background: white;
    border-radius: 15px;
    padding: 2rem;
    margin-top: 2rem;
    box-shadow: 0 8px 25px rgba(0, 0, 0, 0.08);
    border-left: 4px solid #4facfe;
}

/* Image upload area */
.image-upload {
    border: 3px dashed #4facfe;
    border-radius: 15px;
    padding: 3rem;
    text-align: center;
    background: rgba(79, 172, 254, 0.05);
    transition: all 0.3s ease;
}

.image-upload:hover {
    background: rgba(79, 172, 254, 0.1);
    border-color: #00f2fe;
}

/* Progress and loading states */
.progress-bar {
    background: linear-gradient(90deg, #4facfe, #00f2fe);
    height: 4px;
    border-radius: 2px;
    margin: 1rem 0;
}

/* Footer */
.app-footer {
    text-align: center;
    padding: 2rem;
    color: #6c757d;
    border-top: 1px solid #e9ecef;
    margin-top: 3rem;
}

/* Responsive design */
@media (max-width: 768px) {
    .gradio-container {
        margin: 10px;
        border-radius: 15px;
    }
    
    .app-title {
        font-size: 2rem;
    }
    
    .tab-nav button {
        padding: 0.8rem 1.5rem;
        font-size: 0.9rem;
        margin: 0.25rem;
    }
}

/* Dark mode support */
@media (prefers-color-scheme: dark) {
    .input-container, .results-container {
        background: #2d3748;
        color: white;
    }
    
    input[type="text"], textarea {
        background: #4a5568;
        color: white;
        border-color: #718096;
    }
}

/* Animation for smooth transitions */
* {
    transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
}

/* Custom scrollbar */
::-webkit-scrollbar {
    width: 8px;
}

::-webkit-scrollbar-track {
    background: #f1f1f1;
    border-radius: 10px;
}

::-webkit-scrollbar-thumb {
    background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
    border-radius: 10px;
}

::-webkit-scrollbar-thumb:hover {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
}
"""

In [24]:
tab_titles = [
    "🔍 Prompt search",
    "📸 Image search", 
    "🎨 AI image generation"
]


visual_search_webapp = gr.TabbedInterface(
    [webapp_prompt, webapp_image, webapp_genimage],
    tab_titles,
    title="Advanced Visual Search & AI Image Generation Platform with Azure AI",
    css=professional_css,
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="cyan",
        neutral_hue="slate",
        font=gr.themes.GoogleFont("Inter"),
        font_mono=gr.themes.GoogleFont("JetBrains Mono")
    ).set(
        button_primary_background_fill="linear-gradient(135deg, #4facfe 0%, #00f2fe 100%)",
        button_primary_background_fill_hover="linear-gradient(135deg, #667eea 0%, #764ba2 100%)",
        block_radius="15px",
        container_radius="20px"
    )
)

In [25]:
visual_search_webapp.launch(
        share=True,
        show_error=True,
        favicon_path=None,  # Add your favicon path here
        auth=None,  # Add authentication if needed: ("username", "password")
        max_threads=40, # By default, Gradio uses a thread pool (inherited from FastAPI) with a maximum of 40 threads.
        debug=False,
    )

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://313cd7afc0fbeefbd1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Top 01: fashion/0623281002.jpg with Cosine Similarity = 0.61490
Top 02: fashion/0626581003.jpg with Cosine Similarity = 0.61323
Top 03: fashion/0623480002.jpg with Cosine Similarity = 0.61279
Top 01: fashion/0694131021.jpg with Cosine Similarity = 0.62126
Top 02: fashion/0691724010.jpg with Cosine Similarity = 0.61717
Top 03: fashion/0640258011.jpg with Cosine Similarity = 0.61408
Top 01: fashion/0625870004.jpg with Cosine Similarity = 0.62172
Top 02: fashion/0620851002.jpg with Cosine Similarity = 0.60623
Top 03: fashion/0625642002.jpg with Cosine Similarity = 0.60599
Top 01: fashion/0646756002.jpg with Cosine Similarity = 0.61254
Top 02: fashion/0696729002.jpg with Cosine Similarity = 0.60700
Top 03: fashion/0646754002.jpg with Cosine Similarity = 0.60510
Top 01: fashion/0623281002.jpg with Cosine Similarity = 0.61490
Top 02: fashion/0626581003.jpg with Cosine Similarity = 0.61323
Top 03: fashion/0623480002.jpg with Cosine Similarity = 0.61279
Top 01: fashion/0698480005.jpg with Cosi

> Go to next notebook.